In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
df =  pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [3]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)


In [5]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


In [6]:
df.shape


(5572, 3)

In [7]:
df.head()


,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


## Train test split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [9]:
X_train.shape

(4457,)

In [10]:
X_test.shape


(1115,)

In [11]:
type(X_train)

pandas.core.series.Series

In [12]:
X_train[:4]

1639                Great comedy..cant stop laughing da:)
3690                            You still coming tonight?
2475    Princess, i like to make love  &lt;#&gt;  time...
2348                       But i dint slept in afternoon.
Name: Message, dtype: object

In [13]:
type(y_train)

pandas.core.series.Series

In [14]:
y_train[:4]

1639    0
3690    0
2475    0
2348    0
Name: spam, dtype: int64

In [15]:
type(X_train.values)

numpy.ndarray

# Create Bag Of Words Representation Using CountVectorizer

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv


<4457x7791 sparse matrix of type '<class 'numpy.int64'>'
	with 59706 stored elements in Compressed Sparse Row format>

In [17]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0])

In [18]:

X_train_cv.shape

(4457, 7791)

In [19]:

v.get_feature_names_out()[1771]

'chik'

In [20]:

v.vocabulary_

{'great': 3253,
 'comedy': 1899,
 'cant': 1638,
 'stop': 6528,
 'laughing': 4058,
 'da': 2142,
 'you': 7751,
 'still': 6516,
 'coming': 1904,
 'tonight': 7007,
 'princess': 5445,
 'like': 4139,
 'to': 6971,
 'make': 4348,
 'love': 4247,
 'lt': 4271,
 'gt': 3277,
 'times': 6938,
 'per': 5183,
 'night': 4803,
 'hope': 3506,
 'thats': 6851,
 'not': 4857,
 'problem': 5462,
 'but': 1572,
 'dint': 2334,
 'slept': 6261,
 'in': 3658,
 'afternoon': 847,
 'babe': 1185,
 'covers': 2045,
 'your': 7757,
 'face': 2752,
 'kisses': 3973,
 'hi': 3443,
 'frnd': 3027,
 'which': 7535,
 'is': 3768,
 'best': 1321,
 'way': 7454,
 'avoid': 1161,
 'missunderstding': 4537,
 'wit': 7591,
 'our': 5039,
 'beloved': 1311,
 'one': 4971,
 'give': 3160,
 'me': 4428,
 'sec': 6009,
 'breaking': 1488,
 'at': 1110,
 'cstore': 2096,
 'true': 7085,
 'dear': 2193,
 'sat': 5947,
 'pray': 5400,
 'evening': 2672,
 'and': 954,
 'felt': 2823,
 'so': 6322,
 'sms': 6301,
 'some': 6337,
 'time': 6937,
 'working': 7639,
 'technical':

In [21]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0])

In [22]:

np.where(X_train_np[0]!=0)

(array([1638, 1899, 2142, 3253, 4058, 6528]),)

In [23]:
X_train[:4][1579]

KeyError: 1579

In [24]:
X_train_np[0][1771]

0

# Train the naive bayes model

In [25]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [26]:
X_test_cv = v.transform(X_test)


# Evaluate Performance

In [27]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       977
           1       0.99      0.95      0.97       138

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [28]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

## Train the model using sklearn pipeline and reduce number of lines of code

In [29]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [30]:
clf.fit(X_train, y_train)


Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [31]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       977
           1       0.99      0.95      0.97       138

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



# Exercise

- In this Exercise, you are going to classify whether a given movie review is positive or negative.
- you are going to use Bag of words for pre-processing the text and apply different classification algorithms.
- Sklearn CountVectorizer has the inbuilt implementations for Bag of Words.


In [32]:
#Import necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [33]:
#1. read the data provided in the same directory with name 'movies_sentiment_data.csv' and store it in df variable
df = pd.read_csv("movies_sentiment_data.csv")


#2. print the shape of the data
print(df.shape)

#3. print top 5 datapoints
df.head()

(19000, 2)


,review,sentiment
0,"I first saw Jake Gyllenhaal in Jarhead (2005) a little while back and, since then, I've been watching every one of his movies that arrives on my radar screen. Like Clive Owen, he has an intensity (and he even resembles Owen somewhat) that just oozes from the screen. I feel sure that, if he lands some meaty roles, he'll crack an Oscar one day...<br /><br />That's not to denigrate this film at all.<br /><br />It's a fine story, with very believable people (well, it's based upon the author's early shenanigans with rocketry), a great cast  Chris Cooper is always good, and Laura Dern is always on my watch list  with the appropriate mix of humor, pathos, excitement...and the great sound track with so many rock n roll oldies to get the feet tapping.<br /><br />But, this film had a very special significance for me: in 1957, I was the same age as Homer Hickham; like him, I looked up at the night stars to watch Sputnik as it scudded across the blackness; like Homer also, I experimented with rocketry in my backyard and used even the exact same chemicals for fuel; and like Homer, I also had most of my attempts end in explosive disaster! What fun it was...<br /><br />I didn't achieve his great (metaphorical and physical) heights though. But, that's what you find out when you see this movie.<br /><br />Sure, it's a basic family movie, but that's a dying breed these days, it seems. Take the time to see it, with the kids: you'll all have a lot of good laughs.",positive
1,"I enjoyed the movie and the story immensely! I have seen the original(1939 I believe) and enjoyed them both. To really appreciate the story one must be familiar with English culture and customs. The prof.(Peter O'Toole) was dedicated to his school and ""the boys"" in that school. It was an English ""public"" school, which we in the U.S. refer to as a private school (E.G. Andover). He is a very ascetic person and, on the surface, gives the appearance of being stiff, stuffy, uncaring, and weak to the point of being effeminate. He is strict in his educational standards because he DOES care for ""his lads"", i.e., he doesn't want them to get a cheap or weak education. He meets(through introduction) a ""dance hall girl""(Petula Clark) and is totally smitten. In England at the time, the reference to ""dance hall"" carried the connotation of extreme sexual promiscuity and was definitely ""lower class"". We find that the Prof. is in fact a very tough and courageous person as well as loyal to people and institutions that he loves and/or respects. Clark becomes more than a lover and wife...she ""leavens"" his personality and allows him to grow as a man and a person, much to the benefit of his beloved school and his own happiness. The first movie was set BEFORE WW II, this one goes through WW II, also, it is 1969( we've had the ""British Invasion""...Beetles, etc. Clark had hits and was very popular then...still is to me), the music is great, color and photography excellent. I think O'Toole played the character perfectly! There ARE dedicated people like ""Chips""...all around us but many do not receive the recognition. Very enjoyable movie and story!",positive
2,"I had a hard time sitting through this. Every single twist and turn is predictable. You're sitting there just waiting for it ... waiting for it ... and yes, there it is! Just as you predicted at the very beginning of the movie. Or 10 minutes out. Et cetera. <br /><br />Smart writing? No. <br /><br />Torture porn? No, there's no nudity. Other reviews calling this torture porn are most likely written by people on heavy drugs. Unfortunately there's no torture and no nudity (yes, no nudity).<br /><br />There's no suspense at all in this ""thriller"". The only good part about this movie is the ending, but I'm not going to spoil that.<br /><br />I'm giving it a 2/10. A 1/10 would be a horrible B-movie. This movie had better acting.",negative
3,"It's hard to imagine that anyone could find this short their favor

In [34]:
#creating a new column "Category" which represent 1 if the sentiment is positive or 0 if it is negative
df['Category'] = df['sentiment'].apply(lambda x: 1 if x =='positive' else 0)

In [ ]:
#check the distribution of 'Category' and see whether the Target labels are balanced or not.

df['Category'].value_counts()

In [ ]:
#Do the 'train-test' splitting with test size of 20%

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.review, df.Category, test_size=0.2)

# Exercise-1

- using sklearn pipeline module create a classification pipeline to classify the movie review's positive or negative.

- use CountVectorizer for pre-processing the text.

- use Random Forest as the classifier with estimators as 50 and criterion as entropy.

- print the classification report.

In [35]:
#1. create a pipeline object
clf = Pipeline([
    ('vectorizer', CountVectorizer()),                                                    #initializing the vectorizer
    ('random_forest', (RandomForestClassifier(n_estimators=50, criterion='entropy')))      #using the RandomForest classifier
])



#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       977
           1       1.00      0.82      0.90       138

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.98      0.98      1115



# Exercise-2

- using sklearn pipeline module create a classification pipeline to classify the movie review's positive or negative..


- use CountVectorizer for pre-processing the text.
- use KNN as the classifier with n_neighbors of 10 and metric as 'euclidean'.
- print the classification report.

In [36]:
#1. create a pipeline object
clf = Pipeline([
                
     ('vectorizer', CountVectorizer()),   
      ('KNN', (KNeighborsClassifier(n_neighbors=10, metric = 'euclidean')))   #using the KNN classifier with 10 neighbors 
])


#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       977
           1       1.00      0.24      0.39       138

    accuracy                           0.91      1115
   macro avg       0.95      0.62      0.67      1115
weighted avg       0.91      0.91      0.88      1115



# Exercise 3
- use CountVectorizer for pre-processing the text.
- use Multinomial Naive Bayes as the classifier.
- print the classification report.

In [37]:
#1. create a pipeline object
clf = Pipeline([
                
     ('vectorizer', CountVectorizer()),   
      ('Multi NB', MultinomialNB())   #using the Multinomial Naive Bayes classifier 
])


#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       977
           1       0.99      0.95      0.97       138

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



# END